In [27]:
import os, sys, email, datetime, pprint, re, time, html
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [12]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT * FROM mail ORDER BY docno ASC',con=engine, index_col=None)

In [15]:
mail_df.head(10)

,docno,received,isoreceived,sent,isosent,name,email,subject,id,charset,inreplyto,expires,to,cc,body
0,lists-000-13503992,Fri Jul 14 12:48:10 2000,2000-07-14 16:48:10,"Fri, 14 Jul 2000 08:19:41 -0400 (EDT)",2000-07-14 12:19:41,Georgopoulos N,ngeorgopoulos@city.ac.uk,Bug report,E13D4Rs-0004TB-00@mailswitch,US-ASCII,None,-1,html-tidy@w3.org,None,\n\n\n\n\n\n\n\n\n\ntext/enriched attachment: stored\n\n\n\n\n\n\n\n\n
1,lists-001-9920622,Sat Jul 14 07:24:02 2001,2001-07-14 11:24:02,"Sat, 14 Jul 2001 13:12:58 +0200",2001-07-14 11:12:58,Karl Ove Hufthammer,huftis@bigfoot.com,Re: entities in title tags,003b01c10c55$f0f62060$f3468ed5@huftis,iso-8859-1,000201c10c12$b894f900$4222e540@IanEvans,-1,ianevans@digitalhit.com,None,"\n\n\n\n\n\n----- Original Message ----- \n\nFrom: ""Ian M. Evans"" <ianevans@digitalhit.com>\n\nSent: Saturday, July 14, 2001 5:11 AM\n\nSubject: entities in title tags\n\n\n\n\n\n> Just wondering if there's anything wrong with entities in title tags.\n\n\n\nNo.\n\n\n\n> I've noticed that the numeric entity for the apostrophe ends up showing up\n\n> in Google search results as:\n\n\n\nThis is a bug in Google.\n\n\n\n-- \n\nKarl Ove Hufthammer\n\n\n\n\n\n\n"
2,lists-001-9920622,Sat Jul 14 07:24:02 2001,2001-07-14 11:24:02,"Sat, 14 Jul 2001 13:12:58 +0200",2001-07-14 11:12:58,Karl Ove Hufthammer,huftis@bigfoot.com,Re: entities in title tags,003b01c10c55$f0f62060$f3468ed5@huftis,iso-8859-1,000201c10c12$b894f900$4222e540@IanEvans,-1,html-tidy@w3.org,None,"\n\n\n\n\n\n----- Original Message ----- \n\nFrom: ""Ian M. Evans"" <ianevans@digitalhit.com>\n\nSent: Saturday, July 14, 2001 5:11 AM\n\nSubject: entities in title tags\n\n\n\n\n\n> Just wondering if there's anything wrong with entities in title tags.\n\n\n\nNo.\n\n\n\n> I've noticed that the numeric entity for the apostrophe ends up showing up\n\n> in Google search results as:\n\n\n\nThis is a bug in Google.\n\n\n\n-- \n\nKarl Ove Hufthammer\n\n\n\n\n\n\n"
3,lists-002-10134409,Tue Apr 29 10:23:48 2003,2003-04-29 14:23:48,"Tue, 29 Apr 2003 22:23:41 +0800",2003-04-29 14:23:41,lock mode,klcc70@hotmail.com,"Stock markets rise on positive earnings, consumer spending ConsumerReportsorg",Law10-F458juXym3SjK00016b82@hotmail.com,None,None,-1,html-tidy@w3.org,None,"\n\n\n\n\n\n\n\nUnbiased product Ratings from the experts at Product Ratings, product \n\nreviews, buying guides, product safety recalls and consumer information from \n\nthe experts at Consumer Union/Consumer Reports Magazine. Information, \n\nratings, and advice on products, services, and decisions. new car reviews, \n\nused car Consumer Reports Cars and Trucks: Stay informed with our latest new \n\ncar reviews, used car reviews, auto ratings, car safety reports and auto \n\nprice service. Provides car reviews, automobile safety information, car \n\nbuying guidance.\n\n\n\n\n\n\n\nhttp://www.geocities.com/webrobotic2/ConsumerReports.htm\n\n\n\n\n\nconsumer report\n\n12965 consumer report magazine\n\n11859 consumer report car\n\n9172 consumer report online\n\n7784 consumer report auto\n\n4280 consumer report vacuum cleaners\n\n3970 consumer report digital camera\n\n3764 consumer credit report\n\n3633 consumer report automobile\n\n2817 free consumer report\n\n2477 consume..."
4,lists-002-10233238,Sat May 17 03:24:47 2003,2003-05-17 07:24:47,"Wed, 14 May 2003 09:32:46 -0400 (EDT)",2003-05-14 13:32:46,David Van Cott,dtvc17@msn.com,Tag problem,BAY4-F144h4Jzgi1Z8200004355@hotmail.com,None,None,-1,html-tidy@w3.org,dsr@w3.org,\n\n\n\n\n\n\n\n\n\n\n\n\n\nI'm using html tidy at work to make the website more accessible and I'm \n\nhaving a little problem. Everything seems to work great execept for one \n\nthing. Tidy seems to be removing the <dl> tags. This is causing the \n\nalignment of the webpages to be off. I searched through google newsgroups \n\nand the www.w3.org site and I couldnt find any questions or solutions \n\nregarding this. There is my configs...\n\n--clean yes\n\n--output-xml no\n\n--enclose-text yes\n\n--en

In [44]:
for i in mail_df['name'].head(10):
    uri_i = re.sub(r'[^a-zA-Z_0-9]','_',i)
    print(uri_i)

Georgopoulos_N
Karl_Ove_Hufthammer
Karl_Ove_Hufthammer
lock_mode
David_Van_Cott
Azigan
Surfer_s_Choice
Surfer_s_Choice
Surfer_s_Choice
Harold_Baughan__RockSolidSite_com_


## RDFグラフ生成

In [68]:
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
BASE = Namespace("http://www.kde.cs.tsukuba.ac.jp/~aso/trec#")
EMAIL = Namespace("http://www.w3.org/2000/10/swap/pim/email#")

In [69]:
test_instance = mail_df.head(1).values[0]

In [79]:
# RDFぐらふ
g = Graph()
g.bind('',BASE)
g.bind('schema',SDO)
g.bind('email',EMAIL)
g.bind('foaf',FOAF)
g.bind('owl',OWL)
g.bind('rdf',RDF)

In [80]:
docid = test_instance[0]
isoreceived = test_instance[2].isoformat()
isosent = test_instance[4].isoformat()
name = test_instance[5]
email = test_instance[6]
subject = test_instance[7]
uid = test_instance[8]
inreplyto = test_instance[10]
to = test_instance[12] 
cc = test_instance[13]
body = test_instance[14]

#Creating Nodes
##URI参照されるリソースは表記を変換する
emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',uid) 

#tripleを作成
g.add((BASE[emailmessage],RDF.type,SDO.EmailMessage))
g.add((BASE[emailmessage],SDO.identifier,Literal(uid)))
g.add((BASE[emailmessage],SDO.alternateName,Literal(docid)))

if isosent is not None:
    g.add((BASE[emailmessage],SDO.dateSent,Literal(isosent,datatype=XSD.dateTime)))

if isoreceived is not None:
    g.add((BASE[emailmessage],SDO.dateReceived,Literal(isoreceived,datatype=XSD.dateTime)))

if inreplyto is not None:
    reply_emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',inreplyto)
    g.add((BASE[emailmessage],EMAIL.References,BASE[reply_emailmessage]))

if name is not None:
    sender = re.sub(r'[^a-zA-Z_0-9]','_',name)
    g.add((BASE[emailmessage],SDO.sender,BASE[sender]))
    if email is not None:
        sender_email = 'mailto:' + email
        g.add((BASE[sender],FOAF.mbox,URIRef(sender_email)))

if to is not None:
    to_recipient = re.sub(r'[^a-zA-Z_0-9]','_',to)
    to_email = 'mailto:' + to
    g.add((BASE[emailmessage],SDO.toRecipient,BASE[to_recipient]))
    g.add((BASE[to_recipient],FOAF.mbox,URIRef(to_email)))

if cc is not None:
    cc_recipient = re.sub(r'[^a-zA-Z_0-9]','_',cc)
    cc_email = 'mailto:' + cc
    g.add((BASE[emailmessage],SDO.ccRecipient,BASE[cc_recipient]))
    g.add((BASE[cc_recipient],FOAF.mbox,URIRef(cc_email)))

if subject is not None:
    subject = re.sub(r'[^a-zA-Z_0-9]','_',subject)
    g.add((BASE[emailmessage],SDO.headline,Literal(subject)))

if body is not None:
    g.add((BASE[emailmessage],SDO.text,Literal(body)))

In [81]:
print(g.serialize(format="turtle").decode("utf-8"))

@prefix : &lt;http://www.kde.cs.tsukuba.ac.jp/~aso/trec#&gt; .
@prefix foaf: &lt;http://xmlns.com/foaf/0.1/&gt; .
@prefix schema: &lt;https://schema.org/&gt; .
@prefix xsd: &lt;http://www.w3.org/2001/XMLSchema#&gt; .

:E13D4Rs_0004TB_00_mailswitch a schema:EmailMessage ;
    schema:alternateName &quot;lists-000-13503992&quot; ;
    schema:dateReceived &quot;2000-07-14T16:48:10&quot;^^xsd:dateTime ;
    schema:dateSent &quot;2000-07-14T12:19:41&quot;^^xsd:dateTime ;
    schema:headline &quot;Bug_report&quot; ;
    schema:identifier &quot;E13D4Rs-0004TB-00@mailswitch&quot; ;
    schema:sender :Georgopoulos_N ;
    schema:text &quot;&quot;&quot;









text/enriched attachment: stored








&quot;&quot;&quot; ;
    schema:toRecipient :html_tidy_w3_org .

:Georgopoulos_N foaf:mbox &lt;mailto:ngeorgopoulos@city.ac.uk&gt; .

:html_tidy_w3_org foaf:mbox &lt;mailto:html-tidy@w3.org&gt; .


